In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.preprocessing import StandardScaler
#import sklearn


df = pd.read_csv("base_earnings.csv", index_col = 'Unnamed: 0')
df.Date = pd.to_datetime(df.Date)
df.Name = df.Name.str.replace('/d','')
df.Name = df.Name.str.replace(' /d','')

df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['week'] = df['Date'].dt.week
df['day'] = df['Date'].dt.day

#df = df.drop(columns= {'RIC', 'Date'})
df = df.drop(columns= {'RIC', 'Date', 'Industry', 'Sub-sector', 'Sector'})
df.head()

In [ ]:
X=df.drop(['revenue'],axis=1,inplace=False)
y=df.revenue
X

In [ ]:
#df[['Date']].drop_duplicates().sort_values('Date')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df.shape

In [ ]:
pd.value_counts(df.Sector).to_frame().reset_index()

In [ ]:
pd.value_counts(df['Sub-sector']).to_frame().reset_index()

In [ ]:
df.info()

In [ ]:
group1 = df.groupby(['Name'])[['pred_surprise']].agg(np.mean)
group1

In [ ]:
group2 = df.groupby(['Name'])[['pred_surprise']].agg(np.mean)
group2.sort_values('pred_surprise')

In [ ]:
group2 = df.groupby(['Sub-sector'])[['pred_surprise']].agg(np.mean)
group2

In [ ]:
group2 = df.groupby(['Industry'])[['pred_surprise']].agg(np.mean)
group2

In [ ]:
X=df.drop(['mean_estimate'],axis=1,inplace=False)
y=df.mean_estimate
y = y.astype(int)

In [ ]:
num_cols = X.columns[X.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
num_cols

In [ ]:
scaler=StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

In [ ]:
X=pd.get_dummies(X,drop_first=True)
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report,accuracy_score 


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error,confusion_matrix
from sklearn import ensemble, linear_model, neighbors, svm, tree
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn import svm,model_selection, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process


In [ ]:
import seaborn as sns
f,ax = plt.subplots(figsize=(10, 10))
sns.heatmap(X.iloc[:,:7].head().corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
MLA = [
    #Nearest Neighbor
    neighbors.KNeighborsRegressor(),
    #SVM
    svm.SVR(),
    svm.LinearSVR(),
    #Trees    
    tree.DecisionTreeRegressor(),
    #tree.ExtraTreeClassifier(),
    #Linear
    LinearRegression()
    ]

MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)


row_index = 0
for alg in MLA:
    
    print(alg)
    
    predicted = alg.fit(X_train, y_train).predict(X_test)
    print("Done")
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index,'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Train Accuracy'] = round(alg.score(X_train, y_train), 4)
    MLA_compare.loc[row_index, 'MLA Test Accuracy'] = round(alg.score(X_test, y_test), 4)
    
    row_index+=1
    
MLA_compare.sort_values(by = ['MLA Test Accuracy'], ascending = False, inplace = True)    
MLA_compare


#Plot 
plt.subplots(figsize=(15,6))
sns.barplot(x="MLA Name", y="MLA Train Accuracy",data=MLA_compare,palette='hot',edgecolor=sns.color_palette('dark',7))
plt.xticks(rotation=90)
plt.title('MLA Train Accuracy Comparison')
plt.show()

plt.subplots(figsize=(15,6))
sns.barplot(x="MLA Name", y="MLA Test Accuracy",data=MLA_compare,palette='hot',edgecolor=sns.color_palette('dark',7))
plt.xticks(rotation=90)
plt.title('MLA Test Accuracy Comparison')
plt.show()